In [45]:
import pandas as pd
import glob
import os

# === Step 1: Set the folder containing your CSV files ===
folder_path = "/Users/p0s0cad/work/personal-projects/ai-munshi/model/dataset/dataset_templates/"  # <-- change this to your folder path


# === Step 2: Find all CSV files in that folder ===
csv_file_paths = [
    "/Users/p0s0cad/work/personal-projects/ai-munshi/model/dataset/dataset_templates/aggregation_queries_with_df_code.csv",
    "/Users/p0s0cad/work/personal-projects/ai-munshi/model/dataset/dataset_templates/sorting_ranking_queries_with_df_code.csv",
    "/Users/p0s0cad/work/personal-projects/ai-munshi/model/dataset/dataset_templates/text_string_queries_with_df_code.csv"
]

# === Step 3: Read and concatenate them ===
all_dfs = []

for file_path in csv_file_paths:
    df = pd.read_csv(file_path)
    all_dfs.append(df)

final_df = pd.concat(all_dfs, ignore_index=True)

final_df=final_df[["query_type","query","col_1","col_2","df_command"]]

# === Step 4: Save the combined dataset ===
output_path = "./dataset/combined_dataset.csv"
final_df.to_csv(output_path, index=False)

print(f"✅ Combined {len(csv_file_paths)} files into {output_path}")


✅ Combined 3 files into ./dataset/combined_dataset.csv


In [46]:
import random
import pandas as pd
import json

fake_cols_pool = [
    'Request ID', 'Tracking Code', 'Approval Status', 'Internal Notes', 'Timestamp Created',
    'Last Modified By', 'Error Code', 'Sync Status', 'Origin System', 'Archived Flag',
    'Workflow Step', 'Reviewer Comments', 'Flagged Reason', 'Processing Time', 'Manual Override',
    'Document Ref', 'System ID', 'Batch Number', 'Response Time (ms)', 'Audit Trail',
    'Project Tag', 'Release Version', 'Run ID', 'Environment Name', 'Retry Count',
    'Source File Name', 'Alert Triggered', 'Backup ID', 'Session Token', 'Validation Notes'
]


# === Step 1: Load CSV ===
input_csv_path = "./dataset/combined_dataset.csv"  # Replace with your actual file path
df = pd.read_csv(input_csv_path)

# === Step 2: Convert rows to JSONL format ===
output_data = []

for _, row in df.iterrows():
    # Collect columns and build prompt
    columns = list({col.strip() for col in [row["col_1"], row.get("col_2", None)] if pd.notna(col) and col.strip()})
    columns_str = ", ".join(sorted(set(columns)))
    extra_cols = random.sample(fake_cols_pool, 5)
    columns_str = ", ".join(sorted(columns + extra_cols))
    # Create fully delimited Phi-4 style prompt
    print("columns_str:", columns_str)
    prompt = (
        f"<|system|>You are an expert Python assistant. Generate valid Pandas code based on the user's query.\n"
        f"The DataFrame contains the following columns: {columns_str}<|end|>"
        f"<|user|>{row['query']}<|end|>"
        "<|assistant|>"
    )


    # JSON-style assistant completion with <|end|>
    completion = json.dumps({
        "df_code": row["df_command"],
        "query_pred_category": row["query_type"].lower().replace(" ", "_")
    }) + "<|end|>"

    output_data.append({
        "prompt": prompt,
        "completion": completion
    })

# === Step 3: Write to JSONL file ===
output_jsonl_path = "output_phi_4_format.jsonl"
with open(output_jsonl_path, "w") as f:
    for item in output_data:
        f.write(json.dumps(item) + "\n")

print(f"Saved {len(output_data)} items to {output_jsonl_path}")


columns_str: Environment Name, Internal Notes, Origin System, Project Tag, Request ID, channel, quantity
columns_str: Environment Name, Internal Notes, Release Version, Sync Status, Timestamp Created, cost, region
columns_str: Approval Status, Batch Number, Environment Name, Response Time (ms), Session Token, cost, month
columns_str: Approval Status, Error Code, Processing Time, Request ID, Tracking Code, month
columns_str: Audit Trail, Batch Number, Request ID, Sync Status, System ID, profit, store_id
columns_str: Batch Number, Error Code, Processing Time, Project Tag, Run ID, channel, expenses
columns_str: Manual Override, Origin System, Response Time (ms), Source File Name, Sync Status, cost, department
columns_str: Backup ID, Last Modified By, Request ID, Retry Count, Source File Name, category, cost
columns_str: Origin System, Processing Time, Run ID, Timestamp Created, Workflow Step, city, profit
columns_str: Approval Status, Error Code, Internal Notes, Retry Count, Workflow Step

In [ ]:
%pip install transformers==4.51.0
%pip install accelerate==1.6.0
%pip install datasets==3.5.0

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch

# === Configuration ===
model_path = "microsoft/Phi-4-mini-instruct"
dataset_path = "output_phi_4_format.jsonl"
output_dir = "./phi4-full-finetune"
torch.random.manual_seed(0)


# === Load dataset ===
dataset = load_dataset("json", data_files=dataset_path, split="train")

# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token  # phi-4 uses eos for padding

# === Tokenize dataset ===
def tokenize(example):
    full_text = example["prompt"] + example["completion"]
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=1024)

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

# === Load model (full model, no LoRA) ===
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
    cache_dir="/tmp/phi4_cache"
)


/Users/p0s0cad/work/personal-projects/ai-munshi/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.48s/it]


In [ ]:


# === Training Arguments ===
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=10,  # more epochs to help model learn from small data
    logging_steps=5,
    save_strategy="no",
    learning_rate=5e-5,  # slightly higher to help faster adaptation
    weight_decay=0.0,  # unnecessary for tiny datasets
    report_to="none",
    push_to_hub=False
)


# === Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# === Train ===
trainer.train()

# === Save ===
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Full fine-tuning complete. Model saved.")
